# Obtain Seyfert Samples
In this notebook we show the steps to obtain the Seyfert sample of galaxies.

We import only the ``astropy`` Table as most of the work for this step is done outside Jupyter notebooks (with [TOPCAT](http://www.star.bris.ac.uk/~mbt/topcat/)).

In [5]:
from astropy.table import Table

## Obtaining the list

We obtain the [SIMBAD](http://simbad.u-strasbg.fr/simbad/) (SMB) sample of galaxies (SMB_3Dec.fits) with the following SQL query.

**Be aware that here we refer to SQL to the queries. However, [SIMBAD-TAP](http://simbad.u-strasbg.fr/simbad/sim-tap) actually uses [ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language), which is based on SQL.**

~~~~sql
SELECT main_id
	,ra
	,DEC
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

## Using TOPCAT for the cross-match 

We used [TOPCAT](http://www.star.bris.ac.uk/~mbt/topcat/) in this step. First, we obtained the [Veron-Cetty M.P. , Veron P. 2010](https://cdsarc.unistra.fr/viz-bin/cat/VII/258) (VCV) catalog from [VizieR](https://vizier.u-strasbg.fr/viz-bin/VizieR). Then, we added the SMB file in TOPCAT. We cross-matched the two samples in terms of coordinates, assuming galaxies are the same if their coordinates differ by less than 2 arcsecs. The final file (SMB_VCV.fits) is saved and then is transformed in a [VOTable](http://www.star.bris.ac.uk/~mbt/topcat/sun253/inVotable.html) here. This step can also be done in TOPCAT.

In [3]:
Matched = Table.read('../Data/Raw/SMB_VCV.fits', format='fits')
Matched['main_id', 'otype_txt', 'otypes'].write('../Data/Raw/SMBVCV_VO', format='votable')

## Get the bibcodes for the classification type

We run the following SQL query to search for the bibcodes of the otypes in SMB with the SMBVCV_VO file. We save this information in a separated file (BibcodesOtypes.fits)

~~~sql
SELECT Sy.main_id
	,otypes.otype_txt
	,otypes.origin
FROM basic
JOIN ident ON oidref = oid
JOIN otypes USING (oidref)
JOIN TAP_UPLOAD.Seyf AS Sy ON Sy.main_id = basic.main_id
WHERE id = Sy.main_id
~~~

With this information, we can clean the files in 2_Clean_Sample.ipynb

## Another solution
After checking that the two SQL queries can be united without uploading a new table, another solution to get the origin of the otype is run the following SQL query

~~~sql
SELECT DISTINCT main_id
	,ra
	,DEC
	,otypes.otype_txt
	,otypes.origin
FROM basic
JOIN ident ON oidref = oid
JOIN otypes USING (oidref)
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~

and then you could filter with the selected SMB_VCV information. 

##### Notebook info

In [6]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
json    : 2.0.9
numpy   : 1.19.5
autopep8: 1.5.4

Watermark: 2.1.0

